In [18]:
from humanitarian_management_system.models.resourceAllocator import ResourceAllocator
from humanitarian_management_system.models.resourceAdder import ResourceAdder
from humanitarian_management_system.models.resourceReport import ResourceReport

from pathlib import Path
import pandas as pd

In [51]:
df = pd.read_csv("../data/camp.csv")
print(df)

    campID  eventID countryID  refugeeCapacity healthRisk  volunteerPop  \
0        1        3        IN            10000       high             0   
1        2        7        CN               15        low             0   
2        3        7        CN                8       high             0   
3        4        4        IN               20       high             0   
4        5        3        IN               12        low             0   
5        6        7        CN               13        low             2   
6        7        4        IN               15       high             0   
7        8        8        EG               15       high             1   
8        9        8        EG                7       high             1   
9       11        4        IN               10       high             0   
10      12        3        IN              111        low             0   
11      13        5        IN               12       high             0   
12      14        3      

In [58]:
condition = (df['refugeePop']>0) & (df['status']=='open')
valid_open = df.loc[condition, ['campID', 'refugeePop']]
print(valid_open['campID'])

2     3
4     5
8     9
9    11
Name: campID, dtype: int64


In [61]:
resource_stock_csv_path = "../data/resourceStock.csv"
resource_allocation_csv_path = "../data/resourceAllocation.csv"
resource_unallocated_stock_csv_path = "../data/resourceUnallocatedStock.csv"

totalResources_df = pd.read_csv(resource_stock_csv_path)
resourceAllocs_df = pd.read_csv(resource_allocation_csv_path)
unallocResources_df = pd.read_csv(resource_unallocated_stock_csv_path)
joined_df = pd.merge(totalResources_df, resourceAllocs_df, on='resourceID', how='inner') # does not factor in the camps that dont have allocation but have non zero refugees 


In [64]:
print(joined_df)
camps_in_resource_map = joined_df['campID'].unique()

    resourceID                      name  total  priorityLvl  campID    qty
0            1                  Blankets  10084            2       3   3601
1            1                  Blankets  10084            2       5   2881
2            1                  Blankets  10084            2       6   1441
3            1                  Blankets  10084            2       9   1441
4            1                  Blankets  10084            2      11    720
5            2                  Clothing   2850            1       3   1018
6            2                  Clothing   2850            1       5    814
7            2                  Clothing   2850            1       6    407
8            2                  Clothing   2850            1       9    407
9            2                  Clothing   2850            1      11    204
10           3                      Toys     65            3       3     23
11           3                      Toys     65            3       5     19
12          

array([ 3,  5,  6,  9, 11])

In [62]:
resource_camp = joined_df.pivot_table(index=['resourceID', 'name', 'priorityLvl'], columns='campID', values='qty', aggfunc='sum').sort_index(level=0)

joined_df_unalloc_camp = pd.merge(unallocResources_df, resource_camp, on='resourceID', how='inner')


temp = joined_df_unalloc_camp['unallocTotal'].copy()

joined_df_unalloc_camp['unallocTotal'],  joined_df_unalloc_camp['priorityLvl'] = joined_df_unalloc_camp['priorityLvl'], temp
joined_df_unalloc_camp.rename(columns={'unallocTotal': 'priorityLvl', 'priorityLvl': 'unallocTotal'}, inplace=True)

In [67]:
camps_in_resource_map = joined_df['campID'].unique()
valid_camps_without_resources = valid_open[-valid_open['campID'].isin(camps_in_resource_map)]['campID']
print(valid_camps_without_resources)

## if isempty ... 

Series([], Name: campID, dtype: int64)


In [ ]:
# useful validator code

# Define the valid range
valid_range = range(1, 10)  # For example, 1 to 9

while True:
    try:
        # Ask for user input and convert to integer
        user_input = int(input("Enter a number between 1 and 9: "))
        
        # Check if the input is in the valid range
        if user_input in valid_range:
            print("Valid selection!")
            break  # Exit the loop
            
    except:
        # Handles any exception (non-integer input or out-of-range integer)
        print("Invalid selection. Please try again.")


In [19]:
test_instance = ResourceReport()
# trying to use the validator method in resource Report...
valid_range = ['y','n']  # or any list of integers, strings, or a mix of both
error_message = 'Invalid selection. Please try again.'
prompt_message = "Enter a number between 1 and 9, or a specific string: "
test_instance.input_validator(prompt_message, valid_range, error_message)

KeyboardInterrupt: 